# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
base_url = "https://api.geoapify.com/v2/places"
base_url

'https://api.geoapify.com/v2/places'

In [3]:
#curl https://api.radar.io/v1/users \
  #-H "Authorization: prj_live_sk_..."

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,maru,13.0890,5.1628,34.48,27,84,3.35,NG,1696774562
1,1,tara,16.1555,120.4826,25.16,90,12,1.33,PH,1696774565
2,2,we,-29.5369,31.1292,17.21,94,100,3.60,ZA,1696774575
3,3,isafjordur,66.0755,-23.1240,9.02,66,100,7.72,IS,1696774690
4,4,bethel,41.3712,-73.4140,13.97,73,0,7.20,US,1696774690


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
 # Configure the map
Humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
)

# Display the map plot
Humidity_map

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

ideal_weather_city = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5)& (city_data_df['Cloudiness'] ==0)]


ideal_weather_city


# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
104,104,guerrero negro,27.9769,-114.0611,21.02,79,0,1.69,MX,1696774701
153,153,astara,38.5000,48.6667,23.99,70,0,1.65,AZ,1696774705
162,162,khowy,38.5503,44.9521,24.29,31,0,0.00,IR,1696774706
177,177,vallenar,-28.5708,-70.7581,24.62,23,0,2.07,CL,1696774708
181,181,karratha,-20.7377,116.8463,26.52,56,0,1.50,AU,1696774605
187,187,el galpon,-25.3807,-64.6526,26.12,28,0,2.45,AR,1696774709
261,261,pimentel,-6.8367,-79.9342,23.16,68,0,2.06,PE,1696774716
327,327,colonia,50.9333,6.9500,23.75,69,0,1.03,DE,1696774566
449,449,kailua-kona,19.6406,-155.9956,24.17,81,0,3.60,US,1696774651
522,522,illapel,-31.6308,-71.1653,21.99,38,0,2.62,CL,1696774742


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,maru,NG,13.0890,5.1628,27,
1,tara,PH,16.1555,120.4826,90,
2,we,ZA,-29.5369,31.1292,94,
3,isafjordur,IS,66.0755,-23.1240,66,
4,bethel,US,41.3712,-73.4140,73,
...,...,...,...,...,...,...
567,alta,NO,69.9689,23.2717,81,
568,ji'an,CN,27.1172,114.9793,78,
569,mossel bay,ZA,-34.1831,22.1460,68,
570,happy valley-goose bay,CA,53.3168,-60.3315,88,


In [8]:
"https://api.geoapify.com/v2/places=accomadation.hotel"

'https://api.geoapify.com/v2/places=accomadation.hotel'

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row["City"]
    
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
  
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
maru - nearest hotel: Grand ibro oasis hotel
tara - nearest hotel: Barrio Turod
we - nearest hotel: Fairmont Zimbali Resort
isafjordur - nearest hotel: Hótel Horn
bethel - nearest hotel: Hampton Inn Danbury
thompson - nearest hotel: Thompson Inn
talnakh - nearest hotel: Талнах
port alfred - nearest hotel: No hotel found
nabire - nearest hotel: Hotel Anggrek
hawaiian paradise park - nearest hotel: No hotel found
namibe - nearest hotel: Hotel Chik Chik Namibe
brookings - nearest hotel: Quality Inn
alaghsas - nearest hotel: Hôtel Zagado
adamstown - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
grytviken - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
maru - nearest hotel: No hotel found
sint-truiden - nearest hotel: Hôtel Belle-Vie
ola - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
iqaluit - nearest hotel: Frobishe

,City,Country,Lat,Lng,Humidity,Hotel Name
0,maru,NG,13.0890,5.1628,27,Grand ibro oasis hotel
1,tara,PH,16.1555,120.4826,90,Barrio Turod
2,we,ZA,-29.5369,31.1292,94,Fairmont Zimbali Resort
3,isafjordur,IS,66.0755,-23.1240,66,Hótel Horn
4,bethel,US,41.3712,-73.4140,73,Hampton Inn Danbury
...,...,...,...,...,...,...
567,alta,NO,69.9689,23.2717,81,Scandic Alta
568,ji'an,CN,27.1172,114.9793,78,华侨国际酒店
569,mossel bay,ZA,-34.1831,22.1460,68,Protea Hotel Mossel Bay
570,happy valley-goose bay,CA,53.3168,-60.3315,88,Hotel North Two


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=['Country', 'Hotel Name']
    
)

# Display the map plot
map_plot_1

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)